#Basic Data Cleaning Steps

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

In [3]:
df=pd.read_csv(r"C:\Users\abina\Downloads\synthetic_ecommerce_sales_2025\synthetic_ecommerce_sales_2025.csv")

In [4]:
df.head()

,order_id,customer_id,product_category,product_price,quantity,order_date,region,payment_method,delivery_days,is_returned,customer_rating,discount_percent,revenue
0,1,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Beauty,190.40,5,2023-02-21,Europe,BankTransfer,8,0,3.8,0,952.00
1,2,23b8c1e9-3924-46de-beb1-3b9046685257,Fashion,82.22,3,2023-10-13,North America,CreditCard,5,0,3.8,0,246.66
2,3,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,Beauty,15.19,2,2023-06-28,Oceania,Cash,6,1,2.0,10,27.34
3,4,972a8469-1641-4f82-8b9d-2434e465e150,Electronics,310.65,2,2023-07-11,Europe,PayPal,9,0,2.9,5,590.23
4,5,17fc695a-07a0-4a6e-8822-e8f36c031199,Fashion,74.05,4,2023-02-24,Africa,PayPal,3,1,3.1,20,236.96


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          100000 non-null  int64  
 1   customer_id       100000 non-null  object 
 2   product_category  100000 non-null  object 
 3   product_price     100000 non-null  float64
 4   quantity          100000 non-null  int64  
 5   order_date        100000 non-null  object 
 6   region            100000 non-null  object 
 7   payment_method    100000 non-null  object 
 8   delivery_days     100000 non-null  int64  
 9   is_returned       100000 non-null  int64  
 10  customer_rating   100000 non-null  float64
 11  discount_percent  100000 non-null  int64  
 12  revenue           100000 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 9.9+ MB


In [6]:
df.shape

(100000, 13)

In [7]:
df.isnull().sum()

order_id            0
customer_id         0
product_category    0
product_price       0
quantity            0
order_date          0
region              0
payment_method      0
delivery_days       0
is_returned         0
customer_rating     0
discount_percent    0
revenue             0
dtype: int64

#Basic cleaning

In [8]:
#standardizing column names
df.columns=df.columns.str.lower().str.replace(' ','_')
df.columns

Index(['order_id', 'customer_id', 'product_category', 'product_price',
       'quantity', 'order_date', 'region', 'payment_method', 'delivery_days',
       'is_returned', 'customer_rating', 'discount_percent', 'revenue'],
      dtype='object')

In [9]:
#Filling missing discounts
if 'discount' in df.columns:
    df['discount'] = df['discount'].fillna(0)

In [10]:
#date conversion
df['order_date']=pd.to_datetime(df['order_date'])

In [11]:
#numeric_columns_conversion
num_cols = ['product_price','quantity','delivery_days','customer_rating','discount_percent','revenue']
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce')

In [12]:
#Removing invalid transactions
df = df[
    (df['product_price']>0) &
    (df['quantity']>0)&
    (df['revenue']>0)]


In [13]:
#validating discount logic
df = df[(df['discount_percent']>=0) & (df['discount_percent']<=100)]

In [14]:
#Remove duplicates(if any)
df = df.drop_duplicates()

# Removed exact duplicate transactional records to avoid KPI inflation

In [15]:
#Standardizing categorical values
df['product_category'] = df['product_category'].str.title()

In [16]:
df['payment_method'] = df['payment_method'].str.replace(' ','').str.title()

In [17]:
df['region'] = df['region'].str.title()

In [21]:
#Derived Business columns
df['order_value'] = df['product_price'] * df['quantity']

df['net_order_value'] = df['order_value'] * (
    1-df['discount_percent']/100
)

In [24]:
#Final column order
cleaned_df = df[[
    'order_id',
    'order_date',
    'customer_id',
    'product_category',
    'region',
    'payment_method',
    'product_price',
    'quantity',
    'discount_percent',
    'order_value',
    'net_order_value',
    'delivery_days',
    'customer_rating',
    'is_returned'
]]

In [25]:
cleaned_df

,order_id,order_date,customer_id,product_category,region,payment_method,product_price,quantity,discount_percent,order_value,net_order_value,delivery_days,customer_rating,is_returned
0,1,2023-02-21,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Beauty,Europe,Banktransfer,190.40,5,0,952.00,952.0000,8,3.8,0
1,2,2023-10-13,23b8c1e9-3924-46de-beb1-3b9046685257,Fashion,North America,Creditcard,82.22,3,0,246.66,246.6600,5,3.8,0
2,3,2023-06-28,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,Beauty,Oceania,Cash,15.19,2,10,30.38,27.3420,6,2.0,1
3,4,2023-07-11,972a8469-1641-4f82-8b9d-2434e465e150,Electronics,Europe,Paypal,310.65,2,5,621.30,590.2350,9,2.9,0
4,5,2023-02-24,17fc695a-07a0-4a6e-8822-e8f36c031199,Fashion,Africa,Paypal,74.05,4,20,296.20,236.9600,3,3.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,2025-07-21,95898eb0-3712-4824-accb-8d40fab73167,Beauty,Europe,Paypal,354.84,5,0,1774.20,1774.2000,8,3.0,0
99996,99997,2023-02-09,960dc639-d199-4aae-a22e-18d4dfc07403,Automotive,North America,Cash,168.49,2,0,336.98,336.9800,7,4.5,0
99997,99998,2023-06-13,f9897ea5-454f-43c2-a464-2de35502fe95,Electronics,Asia,Paypal,266.82,3,0,800.46,800.4600,5,3.6,0
99998,99999,2024-10-15,17d6608f-5c4b-44da-a4ec-f3171120f7bb,Fashion,North America,Banktransfer,36.09,1,5,36.09,34.2855,6,3.3,1


In [26]:
cleaned_df.to_csv("cleaned_ecommerce_sales_2025.csv",index=False)